# ASR

local ASR

In [12]:
# Initialization
# insanely fast whisper Initialization
import torch
from transformers import pipeline
from transformers.utils import is_flash_attn_2_available

pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-small", # select checkpoint from https://huggingface.co/openai/whisper-large-v3#model-details
    torch_dtype=torch.float16,
    device="cuda:0", # or mps for Mac devices
    model_kwargs={"attn_implementation": "flash_attention_2"} if is_flash_attn_2_available() else {"attn_implementation": "sdpa"},
    generate_kwargs = {"language":"<|en|>","task": "transcribe"},
)


/home/suyash/Desktop/homegrownASR/.conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# Initialization pyaudio for microphone and Webrtc vad

import pyaudio
import webrtcvad
import numpy as np
from pydub import AudioSegment
from io import BytesIO

import time


# Constants
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK_DURATION_MS = 30  # milliseconds
CHUNK_SIZE = int(RATE * CHUNK_DURATION_MS / 1000)  # samples per chunk
SILENCE_THRESHOLD = 100  # adjust this threshold according to your environment
TARGET_DURATION_MS = 500  # milliseconds

def is_silence(chunk):
    return not vad.is_speech(chunk, RATE)

# Initialize PyAudio
audio = pyaudio.PyAudio()

# Initialize WebRTC VAD
vad = webrtcvad.Vad()
vad.set_mode(3)  # Aggressive mode for better voice detection




In [ ]:
# Open stream
stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE,
                     input=True, frames_per_buffer=CHUNK_SIZE)

print("Listening...")

try:
    accumulated_data = b""
    accumulated_silence = 0
    while True:
        chunk = stream.read(CHUNK_SIZE) #read mic data
        is_silent = is_silence(chunk)
        if is_silent:
            accumulated_silence += CHUNK_DURATION_MS
            if accumulated_silence >= TARGET_DURATION_MS:

                if accumulated_data != b"":
                    t0 = time.time()

                    audio_stream = BytesIO(accumulated_data)

                    audio_segment = AudioSegment.from_file(audio_stream, format='raw',
                                        frame_rate=16000, channels=1, sample_width=2)
                    audio_segment.export('output.wav', format='wav')
                    t1 = time.time()

                    outputs = pipe( "output.wav", chunk_length_s=30,batch_size=24,return_timestamps=False,)
                    t2 = time.time()

                    print(outputs)
                    print(t1-t0)
                    print(t2-t1)
                # segments, info = model.transcribe('output.wav', beam_size=5)
                # print(segments)

                    accumulated_data=b""


                #print("No sound detected for {} milliseconds".format(accumulated_silence))
                accumulated_silence = 0  # Reset accumulated silence after printing message
        else:
            accumulated_data += chunk
            accumulated_silence = 0  # Reset accumulated silence if sound is detected
except KeyboardInterrupt:
    pass

# Cleanup
print("Stopping...")
stream.stop_stream()
stream.close()
audio.terminate()

Open ai

In [7]:
import os

from dotenv import load_dotenv
load_dotenv() 


True

In [8]:
from openai import OpenAI
client = OpenAI()

In [14]:
# Initialization pyaudio for microphone and Webrtc vad

import pyaudio
import webrtcvad
import numpy as np
from pydub import AudioSegment
from io import BytesIO

import time


# Constants
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK_DURATION_MS = 30  # milliseconds
CHUNK_SIZE = int(RATE * CHUNK_DURATION_MS / 1000)  # samples per chunk
SILENCE_THRESHOLD = 50  # adjust this threshold according to your environment
TARGET_DURATION_MS = 700  # milliseconds

def is_silence(chunk):
    return not vad.is_speech(chunk, RATE)

# Initialize PyAudio
audio = pyaudio.PyAudio()

# Initialize WebRTC VAD
vad = webrtcvad.Vad()
vad.set_mode(3)  # Aggressive mode for better voice detection




In [16]:
# Open stream
stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE,
                     input=True, frames_per_buffer=CHUNK_SIZE)

print("Listening...")

try:
    accumulated_data = b""
    accumulated_silence = 0
    while True:
        chunk = stream.read(CHUNK_SIZE) #read mic data
        is_silent = is_silence(chunk)
        if is_silent:
            accumulated_silence += CHUNK_DURATION_MS
            if accumulated_silence >= TARGET_DURATION_MS:

                if accumulated_data != b"" :
                    t0 = time.time()

                    audio_stream = BytesIO(accumulated_data)

                    audio_segment = AudioSegment.from_file(audio_stream, format='raw',
                                        frame_rate=16000, channels=1, sample_width=2)
                    audio_segment.export('output.wav', format='wav')
                    audio_file = open("output.wav", "rb")
                    t1 = time.time()

                    transcription = client.audio.transcriptions.create(
                        model="whisper-1", 
                        file=audio_file, 
                        response_format="text"
                        )
                    print(transcription)
                    t2 = time.time()

                    print(t1-t0)
                    print(t2-t1)
                # segments, info = model.transcribe('output.wav', beam_size=5)
                # print(segments)

                    accumulated_data=b""


                #print("No sound detected for {} milliseconds".format(accumulated_silence))
                accumulated_silence = 0  # Reset accumulated silence after printing message
        else:
            accumulated_data += chunk
            accumulated_silence = 0  # Reset accumulated silence if sound is detected
except KeyboardInterrupt:
    pass

# Cleanup
print("Stopping...")
stream.stop_stream()
stream.close()
audio.terminate()

Listening...
हाई सो नॉईस किया या नहीं किया

0.0008525848388671875
2.0941884517669678
Check if the noise went away or not.

0.0009522438049316406
1.3455562591552734
மூடி பார்த்துக்கொண்டு வருகிறேன்

0.0008170604705810547
1.2093002796173096
Stopping...


# Image 

In [17]:
from io import BytesIO
import base64
import pyautogui
from openai import OpenAI 

photo = pyautogui.screenshot()

output = BytesIO()
photo.save(output, format='PNG')
im_data = output.getvalue()

image_data = base64.b64encode(im_data).decode("utf-8")




# Gpt

In [18]:
from io import BytesIO
import base64
import pyautogui
from openai import OpenAI 

photo = pyautogui.screenshot()

output = BytesIO()
photo.save(output, format='PNG')
im_data = output.getvalue()

image_data = base64.b64encode(im_data).decode("utf-8")



QUESTION="Describe the image"
MODEL="gpt-4o"
client = OpenAI()

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant. Ignore the left side of the image and answer about the question asked on the right side of the image"},
        {"role": "user", "content": [
            {"type": "text", "text": QUESTION},
            {"type": "image_url", "image_url": {
                "url": f"data:image/png;base64,{image_data}"}
            }
        ]}
    ],
    temperature=0.0,
)

print(response.choices[0].message.content)

The image is a screenshot of a Visual Studio Code (VS Code) window. The window is displaying a Python script within a Jupyter Notebook file named "final-asr-vad.ipynb". The script appears to be related to taking a screenshot using the `pyautogui` library, encoding the image in base64, and then using the OpenAI API to generate a response.

The VS Code interface includes the following elements:
- The top menu bar with options like File, Edit, Selection, View, Go, Run, Terminal, and Help.
- The notebook file name "final-asr-vad.ipynb" is visible at the top, along with the name of the project "homegrownASR".
- The left sidebar contains icons for various tools and extensions, such as Explorer, Search, Source Control, Run and Debug, Extensions, and others.
- The main editor area is split into two code cells. The first cell contains code for taking a screenshot and encoding it in base64. The second cell, titled "Gpt", contains code for sending a request to the OpenAI API to describe the image

# final

In [ ]:
# Initialization pyaudio for microphone and Webrtc vad

# pyaudio audio
import pyaudio
import webrtcvad
import numpy as np
from pydub import AudioSegment
from io import BytesIO

# Image
import base64
import pyautogui

#GPT and Transcribe
from openai import OpenAI 

import time


# Constants
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK_DURATION_MS = 30  # milliseconds
CHUNK_SIZE = int(RATE * CHUNK_DURATION_MS / 1000)  # samples per chunk
SILENCE_THRESHOLD = 50  # adjust this threshold according to your environment
TARGET_DURATION_MS = 700  # milliseconds

def is_silence(chunk):
    return not vad.is_speech(chunk, RATE)

# Initialize PyAudio
audio = pyaudio.PyAudio()

# Initialize WebRTC VAD
vad = webrtcvad.Vad()
vad.set_mode(3)  # Aggressive mode for better voice detection







from io import BytesIO
import base64
import pyautogui
from openai import OpenAI 


MODEL="gpt-4o"
client = OpenAI()



# Open stream
stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE,
                     input=True, frames_per_buffer=CHUNK_SIZE)

print("Listening...")

try:
    accumulated_data = b""
    accumulated_silence = 0
    while True:
        chunk = stream.read(CHUNK_SIZE) #read mic data
        is_silent = is_silence(chunk)
        if is_silent:
            accumulated_silence += CHUNK_DURATION_MS
            if accumulated_silence >= TARGET_DURATION_MS:

                if accumulated_data != b"" :
                    t0 = time.time()
                    
                    #audio
                    audio_stream = BytesIO(accumulated_data)
                    audio_segment = AudioSegment.from_file(audio_stream, format='raw',
                                        frame_rate=16000, channels=1, sample_width=2)
                    audio_segment.export('output.wav', format='wav')
                    audio_file = open("output.wav", "rb")
                    t1 = time.time()

                    #openai Transcribe
                    transcription = client.audio.transcriptions.create(
                        model="whisper-1", 
                        file=audio_file, 
                        response_format="text"
                        )
                    print("Question = ",transcription)
                    t2 = time.time()




                    # Take Screen Shots
                    photo = pyautogui.screenshot()
                    output = BytesIO()
                    photo.save(output, format='PNG')
                    im_data = output.getvalue()
                    image_data = base64.b64encode(im_data).decode("utf-8")

                    t3= time.time()




                    QUESTION=transcription

                    response = client.chat.completions.create(
                        model=MODEL,
                        messages=[
                            {"role": "system", "content": """You are a helpful assistant. Ignore the left side of the image and use ONLY right side of the image. 
                             HELP the student to THINK. SEE if he is solving the question correctly.
                             Give concise answers not more than 20 words long. Use the image as a reference to check if they are going the right way"""},
                            {"role": "user", "content": [
                                {"type": "text", "text": QUESTION},
                                {"type": "image_url", "image_url": {
                                    "url": f"data:image/png;base64,{image_data}"}
                                }
                            ]}
                        ],
                        temperature=0.0,
                    )

                    t4= time.time()

                    


                    print("GPT answer= ",response.choices[0].message.content)
                # segments, info = model.transcribe('output.wav', beam_size=5)
                # print(segments)
                    print("\n Audio conversion",t1-t0)
                    print("Take Image= ",t3-t2)
                    print("Transcription Time= ",t2-t1)
                    print("Time taken by GPT= ",t4-t3)
                    print("Total Time = ",t4-t0 )

                    accumulated_data=b""


                #print("No sound detected for {} milliseconds".format(accumulated_silence))
                accumulated_silence = 0  # Reset accumulated silence after printing message
        else:
            accumulated_data += chunk
            accumulated_silence = 0  # Reset accumulated silence if sound is detected
except KeyboardInterrupt:
    pass

# Cleanup
print("Stopping...")
stream.stop_stream()
stream.close()
audio.terminate()